In [1]:
import autograd.numpy as np
import pandas as pd

In [2]:
M, N, O = 4, 3, 6
E_np = np.random.randn(M, N, O)

In [3]:
E_np = np.abs(E_np)


In [4]:
E_np = np.divide(E_np, E_np.max())


In [5]:
E_np

array([[[ 0.2495686 ,  0.15639579,  0.19173782,  0.36269131,  0.23746075,
          0.55661969],
        [ 0.31522488,  0.54250973,  0.56055237,  0.41540447,  0.31177043,
          0.13685991],
        [ 0.01019742,  0.24143883,  0.29814897,  0.22976373,  0.02500548,
          0.02241683]],

       [[ 0.33939041,  0.14690375,  0.35667614,  0.30713639,  0.5192242 ,
          0.22449151],
        [ 0.2122677 ,  0.21155024,  0.02749964,  0.03300245,  0.07907378,
          0.23849009],
        [ 0.39319228,  0.02355076,  0.06530103,  0.41411542,  0.12958604,
          0.04898004]],

       [[ 0.197759  ,  0.27113167,  0.80549689,  0.23699048,  0.22752128,
          0.546128  ],
        [ 0.07403889,  0.33653459,  0.18688418,  0.44144554,  0.86480272,
          0.4967378 ],
        [ 0.30985255,  0.48297845,  0.05949094,  0.05040834,  0.16541064,
          0.06481605]],

       [[ 0.20229756,  0.16552947,  0.02749058,  0.20344661,  0.36618726,
          0.25149388],
        [ 0.02746732,  0

In [6]:
E_np_masked = E_np.copy()
for i in range(M):
    for j in range(N):
        for k in range(O):
            if np.random.random()>0.8:
                E_np_masked[i, j, k] = np.NAN

In [7]:
E_np_masked

array([[[        nan,  0.15639579,  0.19173782,  0.36269131,  0.23746075,
          0.55661969],
        [ 0.31522488,  0.54250973,  0.56055237,  0.41540447,  0.31177043,
          0.13685991],
        [        nan,  0.24143883,  0.29814897,  0.22976373,  0.02500548,
          0.02241683]],

       [[ 0.33939041,  0.14690375,  0.35667614,         nan,         nan,
          0.22449151],
        [ 0.2122677 ,  0.21155024,  0.02749964,  0.03300245,  0.07907378,
                 nan],
        [ 0.39319228,  0.02355076,         nan,  0.41411542,  0.12958604,
          0.04898004]],

       [[ 0.197759  ,  0.27113167,         nan,  0.23699048,  0.22752128,
          0.546128  ],
        [ 0.07403889,  0.33653459,  0.18688418,  0.44144554,  0.86480272,
                 nan],
        [        nan,  0.48297845,  0.05949094,  0.05040834,         nan,
          0.06481605]],

       [[ 0.20229756,  0.16552947,  0.02749058,  0.20344661,  0.36618726,
          0.25149388],
        [ 0.02746732,  0

In [10]:
h = 2
t = 3

In [11]:
E_np.shape

(4, 3, 6)

In [12]:
def cost(H, A, T):
    HA = np.tensordot(H, A,axes = [[-1], [0]])
    HAT = np.tensordot(HA, T, axes=[[-1], [0]])
    mask = ~np.isnan(E_np_masked)
    error = (HAT-E_np_masked)[mask].flatten()
    return np.sqrt((error**2).mean())

    

In [13]:
from autograd import multigrad

In [14]:
mg = multigrad(cost, argnums=[0, 1, 2])


In [15]:
lr = 1
H =  np.random.rand(M, h)
A = np.random.rand(h, N, t)
T =  np.random.rand(t, O)

In [19]:
np.allclose(np.tensordot(H, A,axes = [[1], [0]]), np.tensordot(H, A,axes = [[-1], [0]]))

True

In [21]:
HA = np.tensordot(H, A,axes = [[1], [0]])
np.allclose(np.tensordot(HA, T, axes=[[-1], [0]]), np.tensordot(HA, T, axes=[[2], [0]]))

True

In [17]:
A.shape

(2, 3, 3)

In [50]:
for i in range(2000):
    del_h, del_a, del_t = mg(H, A, T)
    H-=lr*del_h
    A-=lr*del_a
    T-=lr*del_t
    H[H<0] = 0
    A[A<0] = 0
    T[T<0] = 0
    if i%500==0:
        print(cost(H, A, T))

0.259098284311
0.145871072094
0.150477191217
0.150144940079


In [48]:
HAT

array([[[ 0.36550529,  0.6042783 ,  0.22580567,  0.343738  ,  0.75256312,
          0.3339246 ],
        [ 0.43228427,  0.9150506 ,  0.32190978,  0.46020755,  0.96733346,
          0.38772689],
        [ 0.39414323,  0.71278863,  0.26969717,  0.49276853,  0.80994261,
          0.39955816]],

       [[ 0.43847886,  0.90903492,  0.3182849 ,  0.42812187,  0.98180905,
          0.38074381],
        [ 0.4402462 ,  0.78562313,  0.29099495,  0.4652659 ,  0.92021891,
          0.4142261 ],
        [ 0.31066988,  0.68949433,  0.23596758,  0.29338615,  0.71841406,
          0.25941337]],

       [[ 0.90821036,  1.6700695 ,  0.60447544,  0.86854758,  1.94229521,
          0.81156645],
        [ 1.00241928,  1.98798063,  0.71274257,  1.06404122,  2.1836946 ,
          0.91681785],
        [ 0.83090865,  1.6195329 ,  0.58997081,  0.95183217,  1.78071561,
          0.79149133]],

       [[ 0.5895851 ,  1.15755277,  0.41130188,  0.57011777,  1.29237337,
          0.51893433],
        [ 0.61951376,  1